In [27]:
import pandas as pd
import itertools
from sklearn.linear_model import ElasticNet
from dateutil.relativedelta import relativedelta
from sklearn.metrics import mean_absolute_error
import numpy as np
%matplotlib inline

In [28]:
data = pd.read_csv('/dsg/demand_anonymized_20170802.csv', delimiter=';', parse_dates=["Month"])
eval = pd.read_csv('/dsg/eval.csv')

In [29]:
# features
clients = data['Ship_To']
cvc = clients.value_counts()
corporate_clients = list(cvc[cvc > 500].index)
regular_clients = list(cvc[cvc <= 500].index)
data['is_corporate'] = data['Ship_To'].isin(corporate_clients)
data['is_regular'] = data['Ship_To'].isin(regular_clients)

In [30]:
oil = pd.read_csv("/dsg/naf.csv", index_col=False, parse_dates=["time"])
oil.columns = ["Oil_price", "Month"]

currency = pd.read_csv("/dsg/currency.csv", index_col=False, parse_dates=["period"])
currency.drop(["Unnamed: 0"], axis=1, inplace=True)
currency.columns = ["curr1", "Month", "curr2"]

ext_features = ['Oil_price', 'curr1', 'curr2']

external_data = oil.merge(currency)
external_data.head()

,Oil_price,Month,curr1,curr2
0,60.51,2015-06-01,1.122559,1.557586
1,56.30,2015-07-01,1.099741,1.556018
2,48.30,2015-08-01,1.113614,1.557757
3,44.45,2015-09-01,1.122852,1.533838
4,44.94,2015-10-01,1.122795,1.534262


In [31]:
# create aggregated by month series
series = data.groupby(["SalOrg", "Material", "Month"]).agg({'OrderQty':'sum', 'is_corporate' : 'sum', 'is_regular' : 'sum'}).reset_index()
series = series.sort_values(by=["SalOrg", "Material", "Month"])
series.head()

,SalOrg,Material,Month,OrderQty,is_regular,is_corporate
0,97LK,00IYcj,2012-05-01,2,1.0,1.0
1,97LK,00IYcj,2012-06-01,13,0.0,2.0
2,97LK,00IYcj,2012-07-01,1,0.0,1.0
3,97LK,00IYcj,2012-09-01,30,0.0,2.0
4,97LK,00IYcj,2012-11-01,1,0.0,1.0


In [32]:
# fill empty
eval_comb = eval[['Material', 'SalOrg']]
eval_comb = list(set([tuple(x) for x in eval_comb.values]))

comb = list(itertools.product(*[eval_comb, list(series['Month'].unique())]))
comb = [(t[0], t[1], m) for t, m in comb]

series2 = pd.DataFrame(comb, columns=['Material', 'SalOrg', 'Month'])
series2 = series2.sort_values(by=['Material', 'SalOrg', 'Month' ])
series2 = series2.merge(series, on=['Month', 'Material', 'SalOrg'], how='left')
series2 = series2.fillna(0)

In [33]:
# add external
series2 = series2.merge(external_data, on='Month', how='left')
series2.head()

,Material,SalOrg,Month,OrderQty,is_regular,is_corporate,Oil_price,curr1,curr2
0,00GB1f,yqSu,2012-01-01,0.0,0.0,0.0,NaN,NaN,NaN
1,00GB1f,yqSu,2012-02-01,0.0,0.0,0.0,NaN,NaN,NaN
2,00GB1f,yqSu,2012-03-01,1.0,0.0,1.0,NaN,NaN,NaN
3,00GB1f,yqSu,2012-04-01,3.0,0.0,2.0,NaN,NaN,NaN
4,00GB1f,yqSu,2012-05-01,2.0,0.0,2.0,NaN,NaN,NaN


In [34]:
series2['OrderQtyLog'] = np.log1p(series2['OrderQty'])

In [35]:
# create lagged features
def lag_feature(df, colname, lag, adv):
    cols_lagged = []
    for i in range(lag, 0, -1):
        col_lagged = '{}(t-{})'.format(colname, i)
        df[col_lagged] = df.groupby(by=['SalOrg', 'Material'])[colname].shift(i)
        cols_lagged.append(col_lagged)
    cols_adv = []
    for i in range(1, adv):
        col_adv = '{}(t+{})'.format(colname, i)
        df[col_adv] = df.groupby(by=['SalOrg', 'Material'])[colname].shift(-i)
        cols_adv.append(col_adv) 
        
    return df, cols_lagged, cols_adv

series, lagged_qty, adv = lag_feature(series2, 'OrderQtyLog', 12, 3)
series, lagged_corp, _ = lag_feature(series, 'is_corporate', 2, 0)
series, lagged_reg, _ = lag_feature(series, 'is_corporate', 2, 0)
lagged = lagged_qty + lagged_corp + lagged_reg
series.head()

,Material,SalOrg,Month,OrderQty,is_regular,is_corporate,Oil_price,curr1,curr2,OrderQtyLog,...,OrderQtyLog(t-6),OrderQtyLog(t-5),OrderQtyLog(t-4),OrderQtyLog(t-3),OrderQtyLog(t-2),OrderQtyLog(t-1),OrderQtyLog(t+1),OrderQtyLog(t+2),is_corporate(t-2),is_corporate(t-1)
0,00GB1f,yqSu,2012-01-01,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.693147,NaN,NaN
1,00GB1f,yqSu,2012-02-01,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.693147,1.386294,NaN,0.0
2,00GB1f,yqSu,2012-03-01,1.0,0.0,1.0,NaN,NaN,NaN,0.693147,...,NaN,NaN,NaN,NaN,0.000000,0.000000,1.386294,1.098612,0.0,0.0
3,00GB1f,yqSu,2012-04-01,3.0,0.0,2.0,NaN,NaN,NaN,1.386294,...,NaN,NaN,NaN,0.0,0.000000,0.693147,1.098612,0.693147,0.0,1.0
4,00GB1f,yqSu,2012-05-01,2.0,0.0,2.0,NaN,NaN,NaN,1.098612,...,NaN,NaN,0.0,0.0,0.693147,1.386294,0.693147,1.386294,1.0,2.0


In [38]:
def create_agg_features(data, columns, name):
    temp = data.loc[(data["Month"] > pd.to_datetime('2014-09-01'))&
                    (data["Month"] < pd.to_datetime('2016-09-01'))].groupby(columns)["OrderQty"].agg(["median", "mean", "std", "min", "max"]).reset_index()
    new_cols = [col + name for col in ["median", "mean", "std", "min", "max"]]
    temp.columns = columns + new_cols
    return temp, new_cols

sm_temp, sm_cols = create_agg_features(series, ["SalOrg", "Material"], "_s_m_")
series = series.merge(sm_temp, how='left', on=["SalOrg", "Material"])

s_temp, s_cols = create_agg_features(series, ["SalOrg"], "_s_")
series = series.merge(s_temp, how='left', on=["SalOrg"])

m_temp, m_cols = create_agg_features(series, ["Material"], "_m_")
series = series.merge(m_temp, how='left', on=["Material"])

series.head()

,Material,SalOrg,Month,OrderQty,is_regular,is_corporate,Oil_price,curr1,curr2,OrderQtyLog,...,median_s_,mean_s_,std_s_,min_s_,max_s_,median_m_,mean_m_,std_m_,min_m_,max_m_
0,00GB1f,yqSu,2012-01-01,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,...,0.0,25.389641,202.069177,0.0,13540.0,0.0,0.26087,0.448978,0.0,1.0
1,00GB1f,yqSu,2012-02-01,0.0,0.0,0.0,NaN,NaN,NaN,0.000000,...,0.0,25.389641,202.069177,0.0,13540.0,0.0,0.26087,0.448978,0.0,1.0
2,00GB1f,yqSu,2012-03-01,1.0,0.0,1.0,NaN,NaN,NaN,0.693147,...,0.0,25.389641,202.069177,0.0,13540.0,0.0,0.26087,0.448978,0.0,1.0
3,00GB1f,yqSu,2012-04-01,3.0,0.0,2.0,NaN,NaN,NaN,1.386294,...,0.0,25.389641,202.069177,0.0,13540.0,0.0,0.26087,0.448978,0.0,1.0
4,00GB1f,yqSu,2012-05-01,2.0,0.0,2.0,NaN,NaN,NaN,1.098612,...,0.0,25.389641,202.069177,0.0,13540.0,0.0,0.26087,0.448978,0.0,1.0


In [36]:
# validation folds
def create_validation(data, date, m):
    return data[(data["Month"] > pd.to_datetime(date) - relativedelta(months=m))&
                (data["Month"] < pd.to_datetime(date))].index, data[(data["Month"] >= pd.to_datetime(date)) & 
                (data["Month"] < pd.to_datetime(date) + relativedelta(months=3))].index

validation_months = ['2016-09-01', '2016-10-01', '2016-11-01']

folds = []
for month in validation_months:
    fold = create_validation(series, month, 3)
    folds.append(fold)

In [11]:
all = len(series.groupby(by=['SalOrg', 'Material']))

In [39]:
errors = {}
all_errors = []
features = lagged + sm_cols + m_cols + s_cols + ['is_regular', 'is_corporate']

for fold in folds:
    fold_errors = []
    
    processed = 1
    for index, group in series.groupby(by=['SalOrg', 'Material']):
        if index not in errors:
            errors[index] = []
        
        train_s = group.loc[group.index.intersection(fold[0])]
        test_s = group.loc[group.index.intersection(fold[1])]
        
        train_y = np.array(train_s[['OrderQtyLog'] + adv])
        train_x = train_s[features]

        test_y = np.array(test_s[['OrderQtyLog'] + adv])
        test_x = test_s[features]

        model = ElasticNet(alpha=3)
        model.fit(train_x, train_y)
        
        test_y_predicted = model.predict(test_x)
        test_y_predicted[test_y_predicted < 0] = 0
        
        error = mean_absolute_error(np.expm1(test_y), np.expm1(test_y_predicted))
        
        errors[index].append(error)
        all_errors.append(error)
        fold_errors.append(error)
        
        processed += 1
        if(processed % 1000 == 0):
            print('Trained {}/{} models, mean fold error: {}'.format(processed, all, np.mean(fold_errors)))
        
    print('Mean fold error: {}'.format(np.mean(fold_errors)))
    
np.mean(all_errors)

Trained 1000/38676 models, mean fold error: 14.588396912988554
Trained 2000/38676 models, mean fold error: 15.745924236816291
Trained 3000/38676 models, mean fold error: 13.07514458334459
Trained 4000/38676 models, mean fold error: 13.669533752389498
Trained 5000/38676 models, mean fold error: 13.473190974503096
Trained 6000/38676 models, mean fold error: 13.899016887421174
Trained 7000/38676 models, mean fold error: 12.987840727510461
Trained 8000/38676 models, mean fold error: 13.968703458540013
Trained 9000/38676 models, mean fold error: 13.628435223666525
Trained 10000/38676 models, mean fold error: 13.344075718208115
Trained 11000/38676 models, mean fold error: 13.182871656375381
Trained 12000/38676 models, mean fold error: 12.733105505998783
Trained 13000/38676 models, mean fold error: 12.165268283021366
Trained 14000/38676 models, mean fold error: 11.911273672137062
Trained 15000/38676 models, mean fold error: 11.923232498689156
Trained 16000/38676 models, mean fold error: 11.66

12.462929214768501

In [20]:
features

['OrderQtyLog(t-12)',
 'OrderQtyLog(t-11)',
 'OrderQtyLog(t-10)',
 'OrderQtyLog(t-9)',
 'OrderQtyLog(t-8)',
 'OrderQtyLog(t-7)',
 'OrderQtyLog(t-6)',
 'OrderQtyLog(t-5)',
 'OrderQtyLog(t-4)',
 'OrderQtyLog(t-3)',
 'OrderQtyLog(t-2)',
 'OrderQtyLog(t-1)',
 'median_s_m_',
 'mean_s_m_',
 'std_s_m_',
 'min_s_m_',
 'max_s_m_',
 'median_m_',
 'mean_m_',
 'std_m_',
 'min_m_',
 'max_m_',
 'median_s_',
 'mean_s_',
 'std_s_',
 'min_s_',
 'max_s_']

In [21]:
models = {}
for index, group in series.groupby(by=['SalOrg', 'Material']):
    train = group.loc[group.index.intersection(fold[1])]

    train_y = train[['OrderQtyLog'] + adv]
    train_x = train[features]

    model = ElasticNet(alpha=5)
    model.fit(train_x, train_y)
    models[index] = model

In [22]:
lagged_final = lagged
lagged_final.append('OrderQtyLog')
lagged_final.pop(0)
fetures_final = lagged_final + sm_cols + m_cols + s_cols
fetures_final

['OrderQtyLog(t-11)',
 'OrderQtyLog(t-10)',
 'OrderQtyLog(t-9)',
 'OrderQtyLog(t-8)',
 'OrderQtyLog(t-7)',
 'OrderQtyLog(t-6)',
 'OrderQtyLog(t-5)',
 'OrderQtyLog(t-4)',
 'OrderQtyLog(t-3)',
 'OrderQtyLog(t-2)',
 'OrderQtyLog(t-1)',
 'OrderQtyLog',
 'median_s_m_',
 'mean_s_m_',
 'std_s_m_',
 'min_s_m_',
 'max_s_m_',
 'median_m_',
 'mean_m_',
 'std_m_',
 'min_m_',
 'max_m_',
 'median_s_',
 'mean_s_',
 'std_s_',
 'min_s_',
 'max_s_']

In [23]:
test_i = series[series["Month"]==pd.to_datetime('2017-03-01')].index

predictions = []
for index, group in series.groupby(by=['SalOrg', 'Material']):
    model = models[index]
    test = group.loc[group.index.intersection(test_i)]
    test_x = test[fetures_final]
    test_prediction = model.predict(test_x)
    prediction = np.expm1(test_prediction)
    prediction[prediction < 0.0] = 0
    row = list(index) + prediction[0].tolist()
    predictions.append(row)
    #print('Prediction done: {}'.format(index))
predictions

[['97LK', '00IYcj', 8.966554934125966, 9.800822982552907, 5.316359597656381],
 ['97LK', '00MFcK', 0.25992104989487314, 0.0, 0.0],
 ['97LK',
  '00Ok8y',
  0.25992104989487314,
  0.25992104989487314,
  0.25992104989487314],
 ['97LK',
  '00W03x',
  0.7099759466766971,
  0.7099759466766971,
  0.7099759466766971],
 ['97LK',
  '00lqzT',
  3.1212852998085556,
  2.8258623655447774,
  2.8258623655447774],
 ['97LK', '00mt9e', 1.75892417638112, 1.41014226417523, 0.25992104989487314],
 ['97LK', '02QhQT', 0.0, 0.0, 0.0],
 ['97LK',
  '02jRc8',
  0.5874010519681994,
  0.5874010519681994,
  0.5874010519681994],
 ['97LK', '0355Np', 0.0, 0.4422495703074083, 0.4422495703074083],
 ['97LK', '03GYN7', 0.0, 0.25992104989487314, 0.25992104989487314],
 ['97LK', '03eCyI', 379.65238810402906, 283.283423993208, 351.24202105790636],
 ['97LK', '041azJ', 1.4101422641752301, 0.0, 0.5874010519681994],
 ['97LK', '04ocTC', 1.0, 1.5198420997897462, 1.8844991406148166],
 ['97LK', '055wMw', 0.0, 0.0, 0.0],
 ['97LK', '05Uaa

In [24]:
pre_df = pd.DataFrame(predictions, columns=['SalOrg', 'Material', '2017-04', '2017-05', '2017-06'])
ml = pd.melt(pre_df, id_vars=['Material', 'SalOrg'], value_vars=['2017-04', '2017-05', '2017-06'])
ml['date'] = ml['variable']
del ml['variable']
ml['demand'] = ml['value']
del ml['value']
ml.head()

,Material,SalOrg,date,demand
0,00IYcj,97LK,2017-04,8.966555
1,00MFcK,97LK,2017-04,0.259921
2,00Ok8y,97LK,2017-04,0.259921
3,00W03x,97LK,2017-04,0.709976
4,00lqzT,97LK,2017-04,3.121285


In [25]:
result = eval.merge(ml, on=['Material', 'SalOrg', 'date'])

In [26]:
result[['ID', 'demand']].to_csv('linear_models_elastic_external_11.95.csv', index=False)